In [6]:
# ==========================================================
# cw_attack_fixed_v4.py
# CW攻撃（L2 Carlini-Wagner）: IMAGENET1K_V1 に統一
# - モデル: MobileNet_V2_Weights.IMAGENET1K_V1
# - mean/std: weights.meta or transforms() 自動取得
# - Foolbox bounds=(0,1)
# - NormalizeModule内包で正規化一貫化（FGSM版と一致）
# - Foolbox v3以降対応 (epsilons=None 必須)
# ==========================================================

import os, re, numpy as np, torch
import torch.nn as nn
import pandas as pd
import torchvision.transforms as T
from torchvision import models
from torch.nn import functional as F
from PIL import Image, UnidentifiedImageError
from pathlib import Path
from tqdm import tqdm
import foolbox as fb

# --- デバイス ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 前処理（Normalizeしない: [0,1] 入力）---
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# --- MobileNetV2 (IMAGENET1K_V1 に統一) ---
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
mobilenetv2 = models.mobilenet_v2(weights=weights).to(device).eval()

# mean/std を meta 優先・なければ transforms() から取得
if hasattr(weights, "meta") and "mean" in weights.meta and "std" in weights.meta:
    MEAN, STD = weights.meta["mean"], weights.meta["std"]
    imagenet_classes = weights.meta.get("categories", None)
else:
    _t = weights.transforms()
    MEAN, STD = _t.mean, _t.std
    imagenet_classes = None

if imagenet_classes is None:
    imagenet_classes = [f"class_{i}" for i in range(1000)]

# --- Normalize 層 ---
class NormalizeModule(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        self.register_buffer("mean", torch.tensor(mean).view(1, 3, 1, 1))
        self.register_buffer("std",  torch.tensor(std).view(1, 3, 1, 1))
    def forward(self, x):
        return (x - self.mean) / self.std

# --- モデル（Normalize を先頭に挿入）---
norm_layer = NormalizeModule(MEAN, STD)
model = nn.Sequential(norm_layer, mobilenetv2).to(device).eval()

# --- Foolbox設定 ---
fmodel = fb.PyTorchModel(model, bounds=(0.0, 1.0))
print("✅ Using MobileNetV2(IMAGENET1K_V1) + mean/std from weights + bounds=(0,1)")

# --- ヘルパー関数群 ---
def decode_img(img_path: str):
    try:
        img = Image.open(img_path).convert("RGB")
        return transform_base(img).unsqueeze(0)  # (1,3,H,W)
    except UnidentifiedImageError:
        print(f"⚠️ 画像を読み込めませんでした: {img_path}")
        return None

def to_torch_clipped(x):
    if isinstance(x, (list, tuple)):
        x = x[0]
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x.clone().detach()
    else:
        raise TypeError(f"Unsupported type: {type(x)}")
    if t.dim() == 3:
        if t.shape[0] in (1,3):
            t = t.unsqueeze(0)
        else:
            t = t.permute(2,0,1).unsqueeze(0)
    elif t.dim() == 2:
        t = t.unsqueeze(0).unsqueeze(0)
    return t.float().cpu().clamp(0, 1)

def save_tensor_image(tensor, path):
    t = tensor.squeeze(0).cpu().clamp(0, 1)
    arr = (t.numpy() * 255).astype(np.uint8)
    arr = np.transpose(arr, (1, 2, 0))
    Image.fromarray(arr).save(path)

def save_noise_vis(orig, clipped, path):
    o = orig.squeeze(0).cpu().numpy()
    c = clipped.squeeze(0).cpu().numpy()
    diff = o - c
    mag = float(np.max(np.abs(diff))) + 1e-8
    vis = (diff / (2 * mag) + 0.5)
    arr = (np.transpose(vis, (1, 2, 0)) * 255).astype(np.uint8)
    Image.fromarray(arr).save(path)

def extract_number(path: Path):
    m = re.search(r'\d+', path.stem)
    return int(m.group()) if m else float('inf')

# --- 推論 ---
def pred(images: torch.Tensor, topk: int = 1):
    with torch.no_grad():
        logits = model(images.to(device))
        probs = F.softmax(logits, dim=1)
        top_probs, top_idxs = torch.topk(probs, topk)
        idx = int(top_idxs[0][0].item())
        return idx, imagenet_classes[idx], float(top_probs[0][0].cpu().item())

def lb_to_tensor(idx: int):
    return torch.tensor([idx], dtype=torch.long, device=device)

# --- CW攻撃（L2）---
def perform_cw_attack(fmodel_local, images, labels, steps=100, binary_search_steps=10, confidence=0.0):
    """Foolbox CW攻撃（epsilons=None を明示指定）"""
    attack = fb.attacks.L2CarliniWagnerAttack(
        steps=steps,
        binary_search_steps=binary_search_steps,
        confidence=confidence
    )
    # ✅ Foolbox v3+ requires epsilons keyword
    raw, clipped, is_adv = attack(fmodel_local, images, labels, epsilons=None)

    if isinstance(clipped, (list, tuple)):
        clipped = clipped[0]
    if clipped is None:
        return None, None, None

    clipped_t = to_torch_clipped(clipped).to(device)
    l2 = torch.norm((images - clipped_t).view(images.shape[0], -1), p=2, dim=1).item()

    adv_idx, _, _ = pred(clipped_t, 1)
    orig_idx, _, _ = pred(images, 1)

    if adv_idx != orig_idx:
        return clipped_t, float(l2), adv_idx
    else:
        return None, float(l2), orig_idx

# --- パス設定 ---
folder_path = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3")
output_adv_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3_CW")
output_noise_dir = output_adv_dir / "noise"
output_adv_tensor_dir = output_adv_dir / "tensor"
for p in [output_adv_dir, output_noise_dir, output_adv_tensor_dir]:
    p.mkdir(parents=True, exist_ok=True)

# --- 攻撃パラメータ ---
cw_steps = 100
cw_binary_search = 10
cw_target_confidence = 0.0

# --- 攻撃実行 ---
AE_results = []
image_files = sorted(
    [f for f in folder_path.iterdir() if f.suffix.lower() in [".jpg", ".jpeg", ".png"]],
    key=extract_number
)

for i, img_path in enumerate(tqdm(image_files, desc="CW Attack")):
    images = decode_img(str(img_path))
    if images is None:
        continue
    images = images.to(device)

    orig_idx, orig_name, orig_conf = pred(images, 1)
    labels = lb_to_tensor(orig_idx)

    clipped, l2, adv_lb = perform_cw_attack(
        fmodel, images, labels,
        steps=cw_steps,
        binary_search_steps=cw_binary_search,
        confidence=cw_target_confidence
    )

    if clipped is not None:
        adv_idx, adv_name, adv_conf = pred(clipped, 1)

        tensor_path = output_adv_tensor_dir / f"{img_path.stem}.pt"
        torch.save(clipped.cpu(), tensor_path)
        save_tensor_image(clipped, output_adv_dir / f"{img_path.stem}.jpg")
        save_noise_vis(images, clipped, output_noise_dir / f"noise_{i}.jpg")

        AE_results.append({
            "image": img_path.name,
            "Original_label": orig_idx,
            "Original_label_name": orig_name,
            "Original_confidence": orig_conf,
            "Advs_label": adv_idx,
            "Advs_label_name": adv_name,
            "Advs_confidence": adv_conf,
            "L2_Distance": l2,
            "Tensor_path": str(tensor_path)
        })
        print(f"[{i}] ✅ 攻撃成功: {orig_name} → {adv_name} (L2={l2:.4f})")
    else:
        print(f"[{i}] ❌ 攻撃失敗: {orig_name} (L2={l2})")

# --- CSV保存 ---
df = pd.DataFrame(AE_results)
out_csv = output_adv_dir / "CW_Results.csv"
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"\n✅ 攻撃結果を保存しました: {out_csv}")


✅ Using MobileNetV2(IMAGENET1K_V1) + mean/std from weights + bounds=(0,1)


CW Attack:   0%|          | 1/1000 [00:02<45:52,  2.76s/it]

[0] ✅ 攻撃成功: class_377 → class_380 (L2=0.2420)


CW Attack:   0%|          | 2/1000 [00:05<46:23,  2.79s/it]

[1] ✅ 攻撃成功: class_701 → class_417 (L2=0.7982)


...

[997] ✅ 攻撃成功: class_877 → class_421 (L2=0.5470)


CW Attack: 100%|█████████▉| 999/1000 [47:12<00:02,  2.91s/it]

[998] ✅ 攻撃成功: class_417 → class_719 (L2=0.8762)


CW Attack: 100%|██████████| 1000/1000 [47:15<00:00,  2.84s/it]

[999] ✅ 攻撃成功: class_89 → class_132 (L2=0.4118)

✅ 攻撃結果を保存しました: C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\CW_Results.csv


In [12]:
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from torchvision import models

# === ImageNetクラス辞書を取得 ===
imagenet_classes = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]
id_to_name = {i: name for i, name in enumerate(imagenet_classes)}

# === CSV読み込み ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_CW\CW_Results.csv"
df = pd.read_csv(csv_path)

# === クラス名（class_xxx → 実名）変換関数 ===
def convert_class_name(value):
    if isinstance(value, str) and value.startswith("class_"):
        try:
            idx = int(value.split("_")[1])
            return id_to_name.get(idx, value)
        except Exception:
            return value
    return value

df["Original_label_name"] = df["Original_label_name"].apply(convert_class_name)
df["Advs_label_name"] = df["Advs_label_name"].apply(convert_class_name)

# === 必要な列を選択 ===
columns = [
    "Original_label_name",
    "Original_confidence",
    "Advs_label_name",
    "Advs_confidence",
    "L2_Distance"
]
df_show = df[columns].copy()

# === 先頭100枚のみ抽出 ===
df_show = df_show.head(100)

# === 桁数調整（すべて小数第6位に丸め） ===
df_show["Original_confidence"] = df_show["Original_confidence"].round(6)
df_show["Advs_confidence"] = df_show["Advs_confidence"].round(6)
df_show["L2_Distance"] = df_show["L2_Distance"].round(6)

# === 日本語列名 ===
df_show.columns = [
    "入力画像ラベル",
    "confidence",
    "敵対的画像ラベル",
    "confidence(敵対的)",
    "L2距離"
]

# === Word文書作成 ===
doc = Document()
doc.add_heading("攻撃結果表（C&W）", level=1)
doc.add_paragraph("C&W攻撃後の入力画像に対する分類結果（先頭100枚）を示す。")

# === 表設定 ===
interval = 100
col_widths = [
    Inches(1.7),
    Inches(1.0),
    Inches(1.7),
    Inches(1.0),
    Inches(0.8)
]

# === 表を出力 ===
for start in range(0, len(df_show), interval):
    subset = df_show.iloc[start:start + interval]
    table = doc.add_table(rows=1, cols=len(df_show.columns))
    table.style = 'Table Grid'

    # --- ヘッダー ---
    hdr_cells = table.rows[0].cells
    for i, col_name in enumerate(df_show.columns):
        p = hdr_cells[i].paragraphs[0]
        run = p.add_run(col_name)
        run.bold = True
        run.font.name = '游明朝'
        run.font.size = Pt(10.5)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # --- データ行 ---
    for _, row in subset.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            if isinstance(val, float):
                text = f"{val:.6f}"
            else:
                text = str(val)
            p = row_cells[i].paragraphs[0]
            run = p.add_run(text)
            run.font.name = '游明朝'
            run.font.size = Pt(10.5)
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # --- 列幅設定 ---
    for row in table.rows:
        for i, width in enumerate(col_widths):
            row.cells[i].width = width

    doc.add_paragraph("")

# === Wordファイル保存 ===
save_path = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_CW\CW_Results_Table_YuMincho_100_English.docx"
doc.save(save_path)

print("✅ C&W結果をImageNet英語クラス名でWord出力しました：", save_path)


✅ C&W結果をImageNet英語クラス名でWord出力しました： C:\Users\sit\wavelet_CGAN\testdata3\data1_CW\CW_Results_Table_YuMincho_100_English.docx
